## CIA Factbook Data Analysis using SQL ##
This DB contains information on countries and their demographics so we can run some analysis and answer a few popualtion related analysis. 
- What countries have areas and population above average?
- Which country will add the most people next year?
- List of countries with higher death rates than birth rates.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [33]:
#Loaded SQL database.

**Looking up the number of tables in the database**

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql 
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Checking for anomalies in Data ###
Writing a query to look for lowest and highest population values, and lowest and highest population growth values.

In [5]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Min population is 0 and Maximum is the entire total world population. Let's look at the names of these areas to make sense of it.

In [16]:
%%sql
SELECT name, MAX(Population)
FROM facts;

 * sqlite:///factbook.db
Done.


name,MAX(Population)
World,7256490011


In [18]:
%%sql
SELECT name, MIN(population)
FROM facts;

 * sqlite:///factbook.db
Done.


name,MIN(population)
Antarctica,0


For accuracy, we will leave out world in query results.

In [20]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [23]:
%%sql
SELECT AVG(population), AVG(area)
FROM facts;

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


## Exploring few population statistics ##

### Looking up countries where population and area are above average. ###

In [26]:
%%sql SELECT name, population, area
FROM facts
WHERE population > (SELECT AVG(population) FROM facts) AND
area > (SELECT AVG(area) FROM facts);

 * sqlite:///factbook.db
Done.


name,population,area
Brazil,204259812,8515770
China,1367485388,9596960
"Congo, Democratic Republic of the",79375136,2344858
Egypt,88487396,1001450
Ethiopia,99465819,1104300
France,66553766,643801
India,1251695584,3287263
Indonesia,255993674,1904569
Iran,81824270,1648195
Mexico,121736809,1964375


### Which country will add the most people next year? ###

In [36]:
%%sql
SELECT name, MAX(population* (population_growth/ 100)) 
FROM facts
WHERE name != 'World';

 * sqlite:///factbook.db
Done.


name,MAX(population* (population_growth/ 100))
India,15270686.124799998


### Which countries have a higher death rate than birth rate? ###

In [32]:
%%sql
SELECT name, birth_rate, death_rate
FROM facts
WHERE birth_rate < death_rate;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


### Country with the highest disparity between births and deaths ###

In [38]:
%%sql
SELECT name, MAX(death_rate - birth_rate)
FROm facts;

 * sqlite:///factbook.db
Done.


name,MAX(death_rate - birth_rate)
Bulgaria,5.52


### Country that will add the least people next year ###

In [43]:
%%sql 
SELECT name, MIN(population * (population_growth/100))
FROm facts
WHERE birth_rate != 'None';

 * sqlite:///factbook.db
Done.


name,MIN(population * (population_growth/100))
Greenland,0.0


The country that will **add the most people** next year is **India**. Country that will add the **least people** next year is **Greenland**. Country with the **largest gap between birth and death rates is Bulgaria**.
